In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

lectures_types = {
    'lecture_id': 'int16',
    'tag': 'int16',
    'part': 'int8',
    'type_of': 'string'
}
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv', dtype = lectures_types) # wszystkie dane ładowane do pamięci
lectures.info()

In [ ]:
train_types = {
    'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'boolean'
}        


train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', dtype = train_types) # wszystkie dane ładowane do pamięci
# train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', dtype = train_types, nrows=1000) # pierwsze 1000 wierszy

In [ ]:
print('W zbiorze istnieje ' + str(lectures.shape[0]) + ' wykładów')

In [ ]:
types = lectures.type_of.value_counts()

fig = plt.figure(figsize=(13,5))
plt.bar(types.index, types.values, color='orange')
plt.xlabel("Typy wykładków")
plt.ylabel("Ilość wystąpień w zbiorze")
plt.show()

In [ ]:
user_lect = train.groupby(["user_id", "answered_correctly"]).size().unstack()
user_lect.columns = ['Wyklad', 'Zle', 'Poprawne']
user_lect['Wyklad'] = user_lect['Wyklad'].fillna(0)
user_lect = user_lect.astype('Int64')
user_lect['Wyklad zaliczony'] = np.where(user_lect['Wyklad'] > 0, True, False)

watches_l = user_lect.groupby("Wyklad zaliczony").agg({'Zle': ['sum'], 'Poprawne': ['sum']})

watches_l['Procent poprawności'] = watches_l['Poprawne'] / (watches_l['Poprawne'] + watches_l['Zle'])

watches_l = watches_l.iloc[:,2]

fig = plt.figure(figsize=(13, 5))
watches_l.plot.bar(color=['red', 'lightgreen'])

plt.xlabel("Użytkowanik brał udział w przynajmniej jednym wykładzie")
plt.ylabel("Procent poprawności odpowiedzi")
plt.xticks(x, labels=['Nie', 'Tak'], rotation=0)
plt.show()

In [ ]:
task_containers = train.groupby(["task_container_id", "answered_correctly"]).size().unstack()
task_containers.columns = ['Wyklad', 'Zle', 'Poprawne']
task_containers = task_containers.astype('Int64')
task_containers['Procent poprawności'] = task_containers['Poprawne'] / (task_containers['Poprawne'] + task_containers['Zle'])
task_containers['Jaki procent wykladow'] = task_containers['Wyklad'] / (task_containers['Wyklad'] + task_containers['Zle'] + task_containers['Poprawne'])
task_containers = task_containers.fillna(0)
task_containers[['Wyklad', 'Procent poprawności', 'Jaki procent wykladow']].sort_values(by="Jaki procent wykladow", ascending=False).head()

In [ ]:
task_containers[['Wyklad', 'Procent poprawności', 'Jaki procent wykladow']].sort_values(by="Wyklad", ascending=False).head()

In [ ]:
task_containers['Byl wyklad'] = np.where(task_containers['Wyklad'] == 0, False, True)
tmp = task_containers.groupby("Byl wyklad").sum()
tmp['Procent poprawności'] = tmp['Poprawne'] / (tmp['Poprawne'] + tmp['Zle'])
tmp = tmp['Procent poprawności']

x = list(tmp.index)
y = list(tmp.values)

fig = plt.figure(figsize = (13, 5)) 
plt.bar(x, y, color=['red', 'lightgreen'])
  
plt.xlabel('Czy w danym task_container był wykład') 
plt.ylabel('Procent poprawności') 
plt.xticks(x, labels=['Nie', 'Tak'])
plt.show() 